In [1]:
# Load modules
import os
import glob
import cftime
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
from tqdm import tqdm
from scipy import signal
import matplotlib.pyplot as plt
import scipy.interpolate as interp

In [2]:
### Manual input ###
# Paths 
dpath     = '/scratch/cimes/GEOCLIM/LRGROUP/datasets/cobalt_data_0406_2020/dep_ESM4_COBALT/'   # to Enhui's ESM4 files 
spath     = '../../forcings/idealized_year_cold_NHtemp/zonal_means/'  # save Zonal Means

# Specify which deposition to prescribe for COBALT
cdep      = ['dep_fed', 'dep_lith', 'dep_po4', 'dep_dry_nh4', 'dep_dry_no3', 'dep_wet_nh4', 'dep_wet_no3']

# Conversion from Kelvin to Celsius
k2C       = -273.15

# Temperature threshold for land mask (in Celsius; threshold does not make a huge difference)
tthresh  = -5

# Strings to load relevant JRA data
str90    = '*_199*'
str20    = '*_20*'
strYmask = '*_2019*'

# Strings for filename
strAvg   = '30yrs'
strReg   = 'basin'
strLand  = 'LandMask'

# Boundaries for averaging time periods (30 years)
t1      = pd.Timestamp(1990, 1, 1, 0)
t2      = pd.Timestamp(2020, 1, 1, 0)

# Time format in deposition files is different
tC1 = cftime.DatetimeNoLeap(1990, 1, 1, 0, 0, 0, 0, has_year_zero=True)
tC2 = cftime.DatetimeNoLeap(2020, 1, 1, 0, 0, 0, 0, has_year_zero=True)

# Boundaries for spatial averaging
# Note: land mask does not work for Africa so stay within ocean
lat1    = 15
lat2    = 65  
lon1    = 295
lon2    = 340

# Zonal mean

In [3]:
### Loop for atmospheric deposition to COBALT ###
for idep in tqdm(cdep):
    dsC = xr.open_dataset(dpath + 'all_' + idep + '_ocean_cobalt_sfc.185001_201412.nc')

    # No need for land mask

    # Calculate zonal average
    zAvg = dsC[idep].sel(yh=slice(lat1, lat2), 
                         xh=slice(lon1-360, lon2-360),  # convert lon
                         time=slice(tC1, tC2)
                        ).mean(dim='xh', skipna=True).compute()

    # Calculate monthly average
    mAvg = zAvg.groupby("time.month").mean(dim="time")

    # Close zAvg (help with memory?) 
    zAvg.close()    
    
    # Save monthly zonal averages for further modifications
    fsname = '_zonal_ESM4_clim' + strAvg + '_monthly_' + strReg + '_' + strLand + '.nc'
    mAvg.to_netcdf(spath + idep + fsname)

    # Close datasets for the forcing
    mAvg.close()
    dsC.close()

100%|██████████| 7/7 [00:07<00:00,  1.04s/it]


# Grid forcing

In [4]:
# Manual input
# Paths
spath     = '../../forcings/idealized_year_cold_NHtemp/zonal_means/'  # to saved zonal means
gpath     = '../../forcings/idealized_year_cold_NHtemp/gridded/'  # save gridded forcing directly into INPUT folder  

# Filenames in and out
fnameC_in   = '_zonal_ESM4_clim30yrs_monthly_basin_LandMask.nc'
fnameC_out  = '_ESM4_clim30yrs_monthly_basin_LandMask_gridded.nc'

# Specify which deposition to grid
cdep     = ['dep_fed', 'dep_lith', 'dep_po4', 'dep_dry_nh4', 'dep_dry_no3', 'dep_wet_nh4', 'dep_wet_no3']

# Lat and lon of forcing domain
hres     = 0.5
lat_out  = np.arange(15, 65.5, hres)  # from 15 to 65
lon_out  = np.arange(300, 350.5, hres)  # from 300 to 350

# Fill value
fillval  = -1e+20  # set fill value (Raphael/Enhui: -1e+20; Elizabeth:-1.e+34)

# Create a 12-month time vector 
time_m = pd.date_range(start = '1990-01-01', periods=12, freq = 'MS')+ pd.DateOffset(days=14)
# Enhui's file has days since 1990-01-01
time_m = np.asarray(time_m.dayofyear, dtype=np.float64)-1

In [5]:
### Function to save gridded forcing ###
# 'ts', saved as 'tas' to be consistent with previous files that required Enhui's Matlab Code
def sforc(gpath, vname, fname_out, lat_out, lon_out, time_out, f_out):
    # Save output - this could be a function earlier in the code
    writing = nc.Dataset(gpath + vname + fname_out, "w", format="NETCDF3_64BIT_OFFSET")
    time    = writing.createDimension("time", None)  # sets unlimited dimension
    yh      = writing.createDimension("yh", len(lat_out))
    xh      = writing.createDimension("xh", len(lon_out))

    yh     = writing.createVariable("yh","f8",("yh"))
    yh[:]  = lat_out
    yh.standard_name    = "latitude"
    yh.long_name        = "latitude"
    yh.units            = "degrees_north"
    yh.axis             = "Y"

    xh     = writing.createVariable("xh","f8",("xh"))
    xh[:]  = lon_out
    xh.standard_name    = "longitude"
    xh.long_name        = "longitude"
    xh.units            = "degrees_east"
    xh.axis             = "X" ;

    time    = writing.createVariable("time","f8",("time"))
    time[:] = time_out
    time.standard_name   = "time"
    time.long_name       = "time"
    time.units           = "days since 1900-01-01 00:00:00" ;
    time.calendar        = "gregorian" ;
    time.axis            = "T" ;
    time.modulo          = " " ;

    var          = writing.createVariable(vname,"f8",("time","yh","xh"),fill_value=fillval)
    var[:,:,:]   = f_out

    # Write information about each variable
    if (vname == 'dep_fed'):
        var.long_name      = "Soluble Wet and Dry dep_fed Deposition to the ocean"
        var.units          = "mol m-2 s-1"
    elif (vname == 'dep_lith'):
        var.long_name      = "Soluble Wet and Dry dep_lith Deposition to the ocean"
        var.units          = "mol m-2 s-1"
    elif (vname == 'dep_po4'):
        var.long_name      = "Soluble Wet and Dry dep_po4 Deposition to the ocean"
        var.units          = "mol m-2 s-1"
    elif (vname == 'dep_dry_nh4'):
        var.long_name      = "Dry Deposition of Ammonia to the ocean"
        var.long_name2     = "Soluble deposition dep_dry_nh4 to the ocean"
        var.units          = "mol m-2 s-1"
    elif (vname == 'dep_dry_no3'):
        var.long_name      = "Dry Deposition of Nitrate to the ocean"
        var.long_name2     = "Soluble deposition dep_dry_no3 to the ocean"
        var.units          = "mol m-2 s-1"
    elif (vname == 'dep_wet_nh4'):
        var.long_name      = "Wet Deposition of Ammonia to the ocean"
        var.long_name2     = "Soluble deposition dep_wet_nh4 to the ocean"
        var.units          = "mol m-2 s-1"
    elif (vname == 'dep_wet_no3'):
        var.long_name      = "Wet Deposition of Nitrate to the ocean"
        var.long_name2     = "Soluble deposition dep_wet_no3 to the ocean"
        var.units          = "mol m-2 s-1"

    writing.close()

In [6]:
def low_filter(data, t_coupe):
    
    sig = data
    fe = 1 # Fréquence d'échantillonnage (yr-1)
    f_nyq = fe / 2.  # Fréquence de nyquist
    fc = 1/t_coupe # Fréquence de coupure (yr-1)
    b, a = signal.butter(4, fc/f_nyq, 'low', analog=False) #filtre de Butterworth en passe-bas
    s_but = signal.filtfilt(b, a, sig) # Application du filtre
    
    return s_but

In [7]:
### Grid deposition zonal means by repeating at each point of longitude ###
for idep in cdep:
    
    # Load appropriate file & create right time vector
    pN = xr.open_dataset(spath + idep + fnameC_in) 
    time_out  = time_m  # using day of year
    fname_out = fnameC_out
    
    # Create a zero array to store the forcing files
    f_out = np.zeros((len(time_out), len(lat_out), len(lon_out)))
             
    # At every time step 
    for itime in np.arange(0,len(time_out)):
        # Interpolate zonal mean to grid latitude
        f0 = np.interp(lat_out, pN.yh, pN[idep][itime,:])
        
        if idep in ['dep_dry_nh4','dep_dry_no3','dep_wet_nh4','dep_wet_no3']:
            f0[lat_out < 80] = 0
        
        if idep in ['dep_fed','dep_lith']:
            f0 = low_filter(f0, t_coupe = 80)
            
        # Repeat the profile everywhere in our domain
        f_out[itime, :, :] = np.tile(f0.reshape(len(lat_out),1), len(lon_out))
    
    # There should not be any NaNs, but replace in case
    f_out[np.isnan(f_out)] = fillval  

    # Save output - this could be a function earlier in the code
    sforc(gpath, idep, fname_out, lat_out, lon_out, time_out, f_out)